<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c26daef747d20be123df43825fb61e4fec5b345cba908359b16886699df56c04
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 12:58:43
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -80.80 K (-0.57%)
Current PnL: -18.92 L (-12.72%)
CY Booked + Current PnL: -7.48 L (-5.03%)
-------------------
Total profit:  1.52 L
Total loss:  -20.43 L
-------------------
Total Booked + Current PnL: 19.50 L (16.03%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.78 L (60.27%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 8.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.46,60.0,H-MC,3.62,195900.0,31230.0,12107.0,-1.89,18.97,6.18,26.31,89.0,2.58,1.39,28.46,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.65,42.0,M-SC,3.09,85170.0,-15607.0,15697.0,-1.01,-15.49,18.43,0.09,245.0,-0.99,0.61,11.89,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.22,110521.0,10017.0,19087.0,-1.14,9.97,17.27,28.96,79.0,0.52,0.79,47.32,MH,ATH,METALS
57,RELIANCE,1533.00,-13.29,49.0,H-LC,4.80,216934.0,6568.0,22214.0,-1.19,3.12,10.24,13.68,37.0,0.30,1.54,20.16,XY25,NTT,REFINERIES
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197580.0,-2558.0,23453.0,-0.71,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.50,85363.0,-28186.0,68239.0,3.49,-24.82,79.94,35.27,268.0,-0.41,0.61,108.93,XR,NTT,HOTELS
7,ATULAUTO,844.0,3808.11,58.0,M-SC,5.32,121776.0,-25592.0,77413.0,2.80,-17.37,63.57,35.16,236.0,-0.33,0.87,24.38,XY24,NTT,AUTO
39,INDUSINDBK,1800.0,246.64,45.0,L-MC,12.29,42562.0,-34644.0,60038.0,1.69,-44.87,141.06,32.89,258.0,-0.58,0.30,17.21,XR,NTT,BANKS
47,LAOPALA,464.0,117.42,54.0,H-SC,2.08,99343.0,-31762.0,83011.0,1.10,-24.23,83.56,39.09,142.0,-0.38,0.71,27.94,AR,NTT,CERAMICS
60,SAIL,228.0,49.16,70.0,M-MC,8.95,235587.0,10625.0,157019.0,0.97,4.72,66.65,74.53,192.0,0.07,1.68,38.50,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-64.90,52.0,H-MC,3.77,249876.0,-51875.0,213294.0,-3.49,-17.19,85.36,53.50,116.0,-0.24,1.78,10.99,XY24,NTT,FMCG
6,ASIANTILES,137.0,6872.22,56.0,L-SC,5.19,76506.0,-17304.0,93919.0,-2.71,-18.45,122.76,81.67,269.0,-0.18,0.54,48.16,XR,NTT,CERAMICS
61,SAMMAANCAP,326.0,-171.59,58.0,M-SC,4.45,82776.0,-19434.0,112824.0,-2.27,-19.01,136.30,91.37,208.0,-0.17,0.59,34.37,XY25,NTT,FINANCE
64,SHALBY,327.0,1226.95,75.0,M-SC,8.07,180416.0,-727.0,46186.0,-1.99,-0.40,25.60,25.10,235.0,-0.02,1.28,44.59,XY24,NTT,HEALTHCARE
45,KANSAINER,340.0,-67.61,54.0,H-SC,0.87,223263.0,-46404.0,82741.0,-1.95,-17.21,37.06,13.47,138.0,-0.56,1.59,12.92,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1226.95,75.0,M-SC,8.07,180416.0,-727.0,46186.0,-1.99,-0.40,25.60,25.10,235.0,-0.02,1.28,44.59,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-20.77,45.0,H-MC,2.07,136255.0,467.0,42280.0,-1.08,0.34,31.03,31.48,107.0,0.01,0.97,12.50,X40,ATH,INSURANCE
23,DABUR,735.00,-7.62,39.0,H-MC,6.89,250018.0,1950.0,106458.0,-1.25,0.79,42.58,43.70,102.0,0.02,1.78,15.35,XY24,BTT,FMCG
29,HAVELLS,2069.16,-1.08,52.0,H-MC,2.02,245372.0,-2359.0,77415.0,-0.53,-0.95,31.55,30.30,92.0,-0.03,1.74,12.21,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.28,53.0,H-MC,3.79,201660.0,840.0,67939.0,-0.66,0.42,33.69,34.25,80.0,0.01,1.43,5.64,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,97.65,42.0,M-SC,3.09,85170.0,-15607.0,15697.0,-1.01,-15.49,18.43,0.09,245.0,-0.99,0.61,11.89,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,9.25,120150.0,-22329.0,41848.0,-0.60,-15.67,34.83,13.70,163.0,-0.53,0.85,41.81,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,54.0,H-SC,0.87,223263.0,-46404.0,82741.0,-1.95,-17.21,37.06,13.47,138.0,-0.56,1.59,12.92,XY24,NTT,PAINTS
18,CERA,9475.0,-21.72,38.0,H-SC,2.10,142796.0,-33107.0,75125.0,-1.48,-18.82,52.61,23.89,149.0,-0.44,1.02,23.32,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.53,41.0,H-MC,7.11,104738.0,-27097.0,69326.0,-1.68,-20.55,66.19,32.03,98.0,-0.39,0.74,17.31,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,70.0,M-MC,8.95,235587.0,10625.0,157019.0,0.97,4.72,66.65,74.53,192.0,0.07,1.68,38.50,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.46,60.0,H-MC,3.62,195900.0,31230.0,12107.0,-1.89,18.97,6.18,26.31,89.0,2.58,1.39,28.46,X40N,NTT,AC
17,CAMS,5211.76,-1.84,57.0,H-SC,2.63,111658.0,9654.0,34268.0,-0.65,9.46,30.69,43.06,122.0,0.28,0.79,28.20,X40N,ATH,MISC
84,VOLTAS,1530.00,-2.06,46.0,H-MC,2.84,205575.0,13833.0,23929.0,-0.41,7.21,11.64,19.69,99.0,0.58,1.46,14.50,XY25,NTT,AC
23,DABUR,735.00,-7.62,39.0,H-MC,6.89,250018.0,1950.0,106458.0,-1.25,0.79,42.58,43.70,102.0,0.02,1.78,15.35,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.22,110521.0,10017.0,19087.0,-1.14,9.97,17.27,28.96,79.0,0.52,0.79,47.32,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.30,47.0,M-SC,1.12,99270.0,7772.0,70978.0,-0.42,8.49,71.50,86.07,223.0,0.11,0.71,44.16,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,68.0,H-MC,7.01,186544.0,6736.0,105677.0,0.63,3.75,56.65,62.52,88.0,0.06,1.33,39.11,XR,NTT,BANKS
86,WIPRO,420.00,-11.52,49.0,M-LC,5.67,155267.0,4322.0,105131.0,0.04,2.86,67.71,72.51,53.0,0.04,1.10,9.06,XR,NTT,IT
37,INDIAMART,4850.92,-53.89,30.0,H-SC,11.03,128636.0,5300.0,128456.0,-0.91,4.30,99.86,108.45,119.0,0.04,0.91,27.61,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.69,35.0,H-LC,7.18,292810.0,-23032.0,135688.0,-1.08,-7.29,46.34,35.67,5.0,-0.17,2.08,5.55,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.92,224274.0,-17650.0,80918.0,-0.79,-7.30,36.08,26.15,8.0,-0.22,1.59,5.98,X40,ATH,IT
31,HINDUNILVR,2922.00,-9.69,40.0,H-LC,10.31,253790.0,-3273.0,38398.0,-1.33,-1.27,15.13,13.67,24.0,-0.09,1.80,18.88,XY25,NTT,FMCG
5,ASIANPAINT,3465.66,-14.37,40.0,H-LC,7.63,216128.0,-35640.0,88850.0,-1.08,-14.16,41.11,21.13,27.0,-0.40,1.54,16.25,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.84,41.0,H-LC,5.30,208884.0,4384.0,37035.0,-1.13,2.14,17.73,20.25,10.0,0.12,1.49,11.06,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.34,50.0,H-LC,1.48,236992.0,-9159.0,88374.0,0.12,-3.72,37.29,32.18,16.0,-0.10,1.69,30.98,X200,ATH,IT
42,ITC,452.0,-38.77,42.0,H-LC,2.04,197580.0,-2558.0,23453.0,-0.71,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,54.0,H-LC,3.33,161910.0,-24185.0,71661.0,-0.16,-13.00,44.26,25.51,15.0,-0.34,1.15,19.32,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-11.28,43.0,H-LC,4.08,273078.0,7652.0,49127.0,-1.79,2.88,17.99,21.40,11.0,0.16,1.94,9.72,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.29,49.0,H-LC,4.80,216934.0,6568.0,22214.0,-1.19,3.12,10.24,13.68,37.0,0.30,1.54,20.16,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287856.0,-58100.0,132961.0,-0.37,-16.79,46.19,21.64,1.0,-0.44,2.05,1.97,X40,ATH,IT
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197580.0,-2558.0,23453.0,-0.71,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.23,55.0,H-LC,7.51,229964.0,-4180.0,52800.0,-0.22,-1.79,22.96,20.77,86.0,-0.08,1.64,5.01,X40N,NTT,BREWERIES
83,VBL,671.64,-18.69,35.0,H-LC,7.18,292810.0,-23032.0,135688.0,-1.08,-7.29,46.34,35.67,5.0,-0.17,2.08,5.55,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.92,224274.0,-17650.0,80918.0,-0.79,-7.30,36.08,26.15,8.0,-0.22,1.59,5.98,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,54.0,H-LC,3.33,161910.0,-24185.0,71661.0,-0.16,-13.00,44.26,25.51,15.0,-0.34,1.15,19.32,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197580.0,-2558.0,23453.0,-0.71,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,41.0,H-LC,5.30,208884.0,4384.0,37035.0,-1.13,2.14,17.73,20.25,10.0,0.12,1.49,11.06,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.37,40.0,H-LC,7.63,216128.0,-35640.0,88850.0,-1.08,-14.16,41.11,21.13,27.0,-0.40,1.54,16.25,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,49.0,H-LC,4.80,216934.0,6568.0,22214.0,-1.19,3.12,10.24,13.68,37.0,0.30,1.54,20.16,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287856.0,-58100.0,132961.0,-0.37,-16.79,46.19,21.64,1.0,-0.44,2.05,1.97,X40,ATH,IT
40,INFY,2275.00,-17.80,50.0,H-LC,8.16,320203.0,7315.0,164360.0,0.25,2.34,51.33,54.87,3.0,0.04,2.28,9.14,X40,BTT,IT
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197580.0,-2558.0,23453.0,-0.71,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
83,VBL,671.64,-18.69,35.0,H-LC,7.18,292810.0,-23032.0,135688.0,-1.08,-7.29,46.34,35.67,5.0,-0.17,2.08,5.55,X40N,ATH,FMCG
1,ABB,7934.00,-39.18,53.0,H-LC,9.04,254275.0,-7344.0,126553.0,-1.22,-2.81,49.77,45.57,7.0,-0.06,1.81,8.20,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6872.22,56.0,L-SC,5.19,76506.0,-17304.0,93919.0,-2.71,-18.45,122.76,81.67,269.0,-0.18,0.54,48.16,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,4.45,82776.0,-19434.0,112824.0,-2.27,-19.01,136.30,91.37,208.0,-0.17,0.59,34.37,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.50,85363.0,-28186.0,68239.0,3.49,-24.82,79.94,35.27,268.0,-0.41,0.61,108.93,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.19,42.0,H-SC,12.05,87290.0,-13673.0,108205.0,0.10,-13.54,123.96,93.63,148.0,-0.13,0.62,30.02,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,49.0,H-SC,12.67,93810.0,-4170.0,25770.0,-0.68,-4.26,27.47,22.05,152.0,-0.16,0.67,36.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1226.95,75.0,M-SC,8.07,180416.0,-727.0,46186.0,-1.99,-0.40,25.60,25.10,235.0,-0.02,1.28,44.59,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3808.11,58.0,M-SC,5.32,121776.0,-25592.0,77413.0,2.80,-17.37,63.57,35.16,236.0,-0.33,0.87,24.38,XY24,NTT,AUTO
71,TANLA,1963.11,-15.81,71.0,H-SC,14.39,225736.0,-45025.0,374970.0,0.35,-16.63,166.11,121.86,133.0,-0.12,1.61,76.59,AR,ATH,IT
81,VAIBHAVGBL,521.00,61.43,62.0,H-SC,6.03,139861.0,-42914.0,153987.0,-1.08,-23.48,110.10,60.77,125.0,-0.28,0.99,25.27,XR,NTT,JEWELLERY
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,4.45,82776.0,-19434.0,112824.0,-2.27,-19.01,136.30,91.37,208.0,-0.17,0.59,34.37,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.68
2,50,74.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.05
LC    32.40
MC    23.55
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.71
X40N     11.41
XR       11.28
XY25     10.03
AR        8.41
OX40N     7.79
X200      1.69
MH        1.65
X5K       1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.40
H-LC    25.91
H-MC    20.59
M-SC    13.14
M-LC     5.45
M-MC     2.66
L-SC     1.51
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.87,-5.13,39.82
IT,12.98,-15.24,78.08
FINANCE,10.07,-15.89,66.45
MISC,7.05,-15.61,79.88
BANKS,6.09,-16.40,77.64
PAINTS,5.63,-16.67,34.06
ELECTRICAL,5.49,-6.66,44.08
INSURANCE,3.70,-6.09,45.75
AC,3.54,3.00,13.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3108453.0,22
XR,1323935.0,14
AR,1148771.0,9
X40,778432.0,10
X40N,602365.0,9
OX40N,549560.0,10
XY25,428067.0,7
SR,248768.0,2
X5K,118772.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658310.0,29
M-SC,1399162.0,17
H-MC,1210213.0,15
H-LC,1177069.0,15
M-LC,382045.0,4
M-MC,304601.0,2
L-SC,248455.0,3
L-MC,60038.0,1
L-LC,38437.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1225244.0      6
M-SC       XY24       798228.0      7
H-SC       AR         785506.0      5
           XR         775096.0      7
H-MC       XY24       557561.0      4
H-LC       X40        490542.0      5
M-MC       XY24       304601.0      2
H-SC       X40N       300450.0      4
           OX40N      259500.0      4
           SR         248768.0      2
H-LC       X40N       225523.0      3
H-MC       X40        215103.0      4
H-LC       AR         198214.0      2
H-MC       XY25       167067.0      2
M-SC       AR         165051.0      2
L-SC       XR         162158.0      2
M-LC       XY24       158142.0      2
M-SC       OX40N      134437.0      4
M-LC       X5K        118772.0      1
M-SC       XR         115835.0      2
           XY25       112824.0      1
H-LC       XY25       109739.0      3
H-MC       XR         105677.0      1
M-LC       XR         105131.0      1
H-LC       X200        88374.0      1
L-SC       OX40N       86297.0      1
H-MC       X40N        76392.0      2
M-SC       X40         72787.0      1
H-MC       OX40N       69326.0      1
H-LC       XY24        64677.0      1
H-SC       MH          63746.0      1
L-MC       XR          60038.0      1
L-LC       XY25        38437.0      1
H-MC       MH          19087.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
